In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

In [ ]:
library(patchwork)
library(scales)
library(ggrepel)
library(ggcorrplot)
library(corrplot)
library(survminer)

In [ ]:
extra_theme <- 
theme(axis.text.x = element_text(angle = 0, size = 12), 
      axis.text.y = element_text(size = 12), 
      plot.title = element_text(size = 16),
      plot.margin = unit(c(1, 1, 1, 0), "cm")) 

# Prepare data for analysis

In [ ]:
base <- readRDS(paste0(SHARE_DIR, "3_marginal_post_process.Rds")) %>% fi(cohortGo != "Pan-Cancer")

In [ ]:
cohorts <- fread(paste0(SHARE_DIR, "top_mechanisms.csv")) %>% fi(cohortGo != "Pan-Cancer")

In [ ]:
sig_fisher_cohorts <- unique(base %>% 
 fi(Odds == "Worse", low_responder, sig_fisher) %>% 
 pu(cohortGo))

In [ ]:
length(base %>% 
 fi(Odds == "Worse", low_responder) %>% 
 se(cohortGo, feature, sig_fisher, sig_pfs) %>% 
 fi(sig_fisher | sig_pfs) %>%
 gb(cohortGo, sig_fisher, sig_pfs) %>% 
 su(ct = n_distinct(feature)) %>%
 fi(sig_fisher) %>% 
 pu(cohortGo) %>% 
 unique())

In [ ]:
base %>% 
 fi(Odds == "Worse") %>% 
 se(cohortGo, feature, sig_fisher, sig_pfs) %>% 
 fi(sig_fisher | sig_pfs) %>%
 su(ct = n_distinct(feature))

# Volcano Plots

- Colors

In [ ]:
options(repr.plot.height = 1, repr.plot.width = 6) 
my_colors <- c("#F04437", "#E81F64", "#903E97", "#65499E", "#4356A5", "#478FCC", "#34A4DD", "#00BCD4", "#009889", "#4BB04F", "#8BC34C", "#CCDA3A", "#FCED3A", "#FFC10E", "#F8991D", "#F1592C", "#7A5649", "#9F9E9E", "#607F8C")
df <- data.frame(color = my_colors,x = seq_along(my_colors))
#ggplot(df, aes(x = factor(x), y = 1, fill = color)) + geom_tile() + scale_fill_identity() +theme_minimal() 

- Color, size, alpha map

In [ ]:
alpha_map <- list("TRUE" = .1, "FALSE" = 1)
size_map <- list("TRUE" = 5, "FALSE" = 2, "Maybe" = 2)
shape_map <- list("Worse" = 25, "Better" = 24)

In [ ]:
fill_map <- 
list(
'Other' = my_colors[18],
'none' = my_colors[18],
'significant' = my_colors[18],
'Anti-PD-1' = my_colors[1], 
'Anti-CTLA-4 / Anti-PD-1' = my_colors[1],    
'Immunotherapy' = my_colors[1], 
'Multikinase inhibitor' = my_colors[10],
'Anti-AR' = my_colors[3],     
'Anti-EGFR' = my_colors[4], 
'Aromatase inhibitor' = my_colors[5],
'Hormonal therapy' = my_colors[5],    
'CAPEOX + Bevacizumab'  = my_colors[6],
'Anti-VEGF / Platinum/Pyrimidine (ant)agonist'  = my_colors[7],   
'Alkaloid / Platinum' = my_colors[8],
'Alkylating / Anthracycline' = my_colors[8],    
'Platinum / Pyrimidine (ant)agonist'= my_colors[9],
'Platinum / Taxane'= my_colors[10],
'Pyrimidine (ant)agonist'= my_colors[11],
'Taxane'= my_colors[12],
'Chemotherapy' = my_colors[8], 
'Targeted therapy' = my_colors[10]) 

In [ ]:
color_map <- list()
for( i in names(fill_map)){
 if( i %in% c("other", "none", "significant")){
    color_map[[i]] <- my_colors[18]
 } else {
    color_map[[i]] <- "black"
 }
}

- Make the plot

In [ ]:
main_figure <- 
 base %>% 
   ar(desc(Treatment)) %>% 
   ggplot( 
    aes(x = prob_response, 
        y = -log10(p_fdr_fisher),
        alpha = highlight, 
        color = sig_fisher,
        fill = Treatment, 
        size = highlight, 
        shape = Odds)) + 
   geom_point() + 
   scale_alpha_manual(values = unlist(list("TRUE" = 1, "FALSE" = .15))) + 
   scale_color_manual(values = unlist(list("TRUE" = "black", "FALSE" = "white"))) +  
   scale_size_manual(values = unlist(size_map)) + 
   scale_fill_manual(values = unlist(fill_map)) +  
   scale_shape_manual(values = unlist(shape_map)) +   
   go_theme + 
   extra_theme + 
   geom_hline(yintercept = 1, alpha = .7, color = "red") + 
   geom_vline(xintercept = .1, alpha = .7, size = .1) + 
   labs(x = "Estimated Probability of Response", y = "Fisher's Test\n-Log10 (FDR Adjusted p-value)", title = "Systematic Analysis Results") + 
   guides(alpha = "none", 
          size = "none", 
          color = "none", 
          shape = guide_legend(override.aes = list(size = 4)),
          fill = guide_legend(override.aes = list(shape = 21, size = 4))
         ) + 
   scale_x_continuous( labels = percent_format(accuracy = 1), limits = c(0,1))

- Main Figures

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
main_figure

- Zoomed figures

In [ ]:
zoom_df <-  base %>% ar(desc(Treatment)) %>% fi(highlight)

In [ ]:
show <- 
data.frame(
 cohortGo = c("Prostate / Abiraterone", 
              "Bladder Urothelial / Anti-PD-1", 
              "Skin Melanoma / Immunotherapy", 
              "Colorectum / Chemotherapy", 
              "Pan-Cancer / Pazopanib", 
              "Lung NSCLC / Anti-PD-1",
              "Pan-Cancer / Alkylating / Anthracycline",
              "Pan-Cancer / Pazopanib",
              "Lung NSCLC / Immunotherapy",
              "Pan-Cancer / Immunotherapy"),      
 feature = c("rna_geneset_KEGG_TGF_BETA_SIGNALING_PATHWAY_gt25", 
             "rna_geneset_HALLMARK_HEDGEHOG_SIGNALING_gt50", 
             "drivers_pathway_IMMUNE_EVASION_gt0", 
             "hotspot_KRAS_position_25398284", 
             "rna_geneset_KEGG_ARGININE_AND_PROLINE_METABOLISM_lt50", 
             "neo_ct_lt75",
             "rna_geneset_KEGG_PROPANOATE_METABOLISM_gt75", 
             "rna_geneset_KEGG_HISTIDINE_METABOLISM_lt50",
             "purity_tmbPerMb_lt6",
             "rna_geneset_KEGG_TAURINE_AND_HYPOTAURINE_METABOLISM_gt75"))

In [ ]:
zoom_in <- zoom_df %>% ij(show, by = c("cohortGo", "feature"))

In [ ]:
highlight <- 
main_figure + 
 scale_x_continuous( labels = percent_format(accuracy = 1), limits = c(-.003,.103))  + 
 ylim(1,1.67) + 
 theme(axis.title.y = element_blank()) + 
 labs(x = "Estimated Probability of Response", 
      y = "Fisher's Test\n-Log10 (p-value)", 
      title = "Highlighted Univariate Results", size = NULL, alpha = NULL, color = NULL) +
 geom_point(size = 7) + 
 geom_text_repel(data = zoom_df, 
                 aes(label = paste0(cohortGo, "\n", clean_name) ), color = "black", alpha = 1, size = 2, nudge_y = .07,
                    force = 2,               # increase repulsion force (default = 1)
                    force_pull = 0.1,        # reduce attraction toward anchor point (default = 0.1)
                    box.padding = 0.5 ) 

# Together

In [ ]:
go <- (main_figure + theme(legend.position = "none") | highlight ) + plot_layout(widths = c(6, 6))

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 6)
go
ggsave( paste0(SHARE_DIR, "1_volcano_main.png"), plot = go, width = 14, height = 6)

# Highlight Top with interaction

## First Highlight in volcano plot

In [ ]:
interaction_base <- 
base %>% 
 fi(cohortGo != "Pan-Cancer") %>% 
 drop_na(p_fdr_surv) %>% 
 mu(focus = ifelse(p_fdr_fisher < .01 & p_fdr_surv < .01 & Odds == "Worse" & mechanism == "Anti-PD-1", TRUE, FALSE),
    `PFS Significant` = (p_fdr_surv < .01), 
    Treatment = ifelse(focus, mechanism, "Other"))

In [ ]:
highlight_interaction_figure <- 
interaction_base %>% 
   ar(desc(Treatment)) %>% 
   ggplot( 
    aes(x = prob_response, 
        y = -log10(p_fdr_fisher),
        alpha = focus, 
        color = `PFS Significant`,
        fill = Treatment, 
        size = focus, 
        shape = Odds)) + 
   geom_point() + 
   scale_alpha_manual(values = unlist(list("TRUE" = 1, "FALSE" = .15))) + 
   scale_color_manual(values = unlist(list("TRUE" = "black", "FALSE" = "white"))) +  
   scale_size_manual(values = unlist(size_map)) + 
   scale_fill_manual(values = unlist(fill_map)) +  
   scale_shape_manual(values = unlist(shape_map)) +   
   go_theme + 
   extra_theme + 
   geom_hline(yintercept = 2, alpha = .7, color = "red") + 
   geom_vline(xintercept = .1, alpha = .7, size = .1) + 
   labs(x = "Estimated Probability of Response", y = "Fisher's Test\n-Log10 (p-value FDR Adjusted)", title = "Systematic Analysis Results") + 
   guides(alpha = "none", 
          size = "none",  
          shape = guide_legend(override.aes = list(size = 4)),
          fill = guide_legend(override.aes = list(shape = 21, size = 4))
         ) + 
   scale_x_continuous( labels = percent_format(accuracy = 1), limits = c(0,1))

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
highlight_interaction_figure

## Highlight Correlations of Top Features

In [ ]:
cohort <- "Pan-Cancer / Anti-PD-1"

In [ ]:
fisher_base <- fread(paste0(SHARE_DIR, "fisher_base.csv"))

In [ ]:
highlights <- interaction_base %>% fi(cohortGo == cohort, focus, !grepl("vhio", feature)) 

- Correlation Figure

In [ ]:
tmp <-
fisher_base %>% 
 fi(cohortGo == "Pan-Cancer / Anti-PD-1") %>% 
 se( any_of(highlights %>% pu(feature)))

In [ ]:
replacements <- 
c("_" = " ", 
  "rna geneset " = "RNA", 
  "gene set" = "",
  "HALLMARK" = "", 
  "KEGG" = "",
  "gt0" = "",
  "gt75" = "Very High",
  "gt50" = "High",
  "gt25" = "Mod/High",
  "lt75" = "Low/Mod",
  "lt50" = "Low",
  "lt25" = "Very Low",
  "purity tmbStatus" = "TMB",
  "hotspot KRAS position 25398284" = "KRAS G12D hotspot",
  "purity tmbPerMb lt6" = "TMB per MB < 6",
  "neo ct" = "RNA Neoantigens",
  "TGF BETA SIGNALING PATHWAY" = "TGFB",
  "purity tmlStatus low" = "TML Low",
  "purity tmbPerMb lt4" = "TMB per MB < 4",
  "RNAAPM" = "RNA APM",
  "low" = "Low",
  "SIGNALING " = "",
  "t cell" = "T-cell",
  "RENIN ANGIOTENSIN SYSTEM " = str_to_title("RENIN ANGIOTENSIN SYSTEM "),
  " AND " = "/")

# Loop over names and replace
s1 <- highlights %>% pu(feature)
for (pattern in names(replacements)) {
  s1 <- gsub(pattern, replacements[pattern], s1)
}
names(tmp) <- s1

In [ ]:
cor_base <- cor( tmp,use = "pairwise.complete.obs")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
correlations <- 
ggcorrplot(cor_base, 
           method = "circle",   # "circle", "square", "lower", "upper"
           insig = "blank",     # hide insignificant correlations
           tl.cex = 8) + 
 scale_x_discrete(guide = guide_axis(n.dodge = 6)) + 
 theme(axis.text.x = element_text(angle = 0, vjust = 1, hjust = 0.5)) +
 ggtitle("Correlation of Highly Significant Features")

## Survival Analyses

In [ ]:
tmp <-
fisher_base %>% 
 fi(cohortGo == "Pan-Cancer / Anti-PD-1") %>% 
 se( daysToPfsEvent, pfsEvent, non_response, any_of(highlights %>% pu(feature)))

In [ ]:
pfs_base <-
tmp %>% 
 se(daysToPfsEvent, pfsEvent, purity_tmbStatus_low, 
    rna_geneset_gene_set_t_cell_effector_lt75, 
    rna_geneset_KEGG_RENIN_ANGIOTENSIN_SYSTEM_gt50, 
    non_response) %>%
 mu( event = rna_geneset_KEGG_RENIN_ANGIOTENSIN_SYSTEM_gt50 + rna_geneset_gene_set_t_cell_effector_lt75 + 
     purity_tmbStatus_low) %>% 
 se(daysToPfsEvent, pfsEvent, non_response, event) %>% 
 drop_na(event) %>% 
 gb(event) %>% 
 mu(pct_response = mean(1-non_response)) %>% 
 mu(Events = as.factor(paste0(event, " (", 100*round(pct_response,2), "% response)")))

In [ ]:
#pfs_base %>% gb(event, non_response) %>% su(ct = n())

In [ ]:
surv_formula <- expr(Surv(daysToPfsEvent, pfsEvent) ~ Events)
fits <- survfit(eval(surv_formula, envir = environment()), data = pfs_base)
pval <- signif(survdiff(eval(surv_formula, envir = environment()), data = pfs_base)$pvalue,2)

In [ ]:
#fit
oo <- 
ggsurvplot(
 fits,
 data = pfs_base,
 #palette = c("#7AABD3", "#e52f28", "#7AABD3", "#e52f28"),
 conf.int = TRUE, 
 risk.table = TRUE, 
 pval.coord = c(700, .95), 
 xlim = c(0, 365*4), 
 break.time.by = 300, 
 ggtheme = theme_minimal(),
 xlab = "Days", 
 ylab = "Survival Probability", 
 title = "Combination Marker of TMB, T-cell, Renin Angiotensin" ) 

oo$plot <- 
oo$plot + 
 annotate("text", x = 1000, y = 0.9, label = paste0("Log-rank        \np-value = ", pval), size = 5) + 
 theme(plot.title = element_text(hjust = 0.5)) + 
 guides(color = guide_legend(nrow = 2, byrow = TRUE)) 

pfs_plot <- oo$plot #+ guides(colour = guide_legend(nrow = 1)) #as_ggplot(gridExtra::arrangeGrob(oo$plot, oo$table, layout_matrix = matrix(c(1,1,1,1,1,1,1,2,2)))) 

## Combine into one figure

In [ ]:
interaction_go <- (highlight_interaction_figure +pads | correlations | pfs_plot ) + plot_layout(widths = c(1, 1, 1))

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 7)
interaction_go

In [ ]:
ggsave( paste0(SHARE_DIR, "4_combination_marker.png"), plot = interaction_go, width = 20, height = 7)

In [ ]:
 paste0(SHARE_DIR, "4_combination_marker.png")